In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import plotly.express as px

In [2]:
# CEL sum
#RELU 150 259
#sigmoid 146 231
#leaky_relu 110 160

In [3]:
class Perceptron(nn.Module):
    """Perceptron.
    
    The perceptron network consists of a single layer of [output_size] perceptron neurons connected to [input_size] inputs through a set of weights and biases.
    """
    def __init__(self, input_size, output_size):
        """
        Keyword arguments:
        input_size -- the number of columns of the input tensor
        output_size -- the number of columns of the output tensor
        """
        super(Perceptron, self).__init__()
        self.layer = nn.Linear(input_size, output_size)
        
    @torch.no_grad()
    def init_weights(self):
        nn.init.xavier_uniform(self.layer.weight)
        self.layer.bias.data.fill_(0.01)

    def forward(self, x):
        return F.leaky_relu(self.layer(x))

In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, target_column=None):
        if target_column:
            self.data, self.y  = dataframe.drop(columns=target_column), dataframe[target_column]
        else:
            self.data, self.y  = dataframe, [np.nan,]*self.__len__()
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        return self.data.iloc[index].to_numpy(np.float32), self.y.iloc[index]

In [13]:
BATCH_SIZE = 100
EPOCHS = 10000
lr = 3e-3

In [6]:
X = pd.read_csv('./data/winequality-red.csv')
X.quality = X.quality - 3
data_tr, data_va = train_test_split(X, test_size=0.1, shuffle=True)
train_dataset = CustomDataset(data_tr, target_column='quality')
valid_dataset = CustomDataset(data_va, target_column='quality')
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
valid_loader = DataLoader(train_dataset, batch_size=len(valid_dataset))

In [7]:
model = Perceptron(11, 6)

In [8]:
criterion = nn.CrossEntropyLoss(reduction='sum')

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
# !mkdir ./logs
train_writer = SummaryWriter('./logs/train/')
valid_writer = SummaryWriter('./logs/valid/')

In [11]:
step = 0

In [15]:
for i in range(EPOCHS):
    model.train()
    for data, label in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        step += 1
        if not step % 10:
            train_writer.add_scalar('loss', loss, global_step=step)
    model.eval()
    with torch.no_grad():
        for data, label in valid_loader:
            output = model(data)
            loss = criterion(output, label)
        valid_writer.add_scalar('loss', loss, global_step=step)

KeyboardInterrupt: 

In [16]:
model.eval()
with torch.no_grad():
    for data, label in valid_loader:
        output = model(data)

In [24]:
np.argmax(output,axis=1)

tensor([2, 2, 3, 2, 3, 2, 4, 0, 2, 2, 3, 4, 1, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2,
        2, 3, 2, 4, 2, 3, 4, 3, 2, 2, 3, 2, 3, 2, 2, 1, 3, 5, 2, 2, 2, 3, 3, 2,
        4, 2, 4, 3, 3, 2, 1, 3, 3, 3, 4, 4, 3, 3, 3, 3, 2, 3, 3, 3, 2, 4, 4, 3,
        2, 5, 4, 4, 2, 2, 3, 4, 4, 3, 3, 3, 4, 2, 4, 3, 2, 2, 4, 4, 2, 3, 3, 2,
        3, 3, 3, 4, 4, 2, 2, 3, 2, 4, 3, 2, 2, 5, 3, 3, 3, 3, 2, 2, 3, 2, 1, 2,
        3, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 4, 2, 2, 1, 3, 2, 2,
        2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2])

In [25]:
label

tensor([2, 2, 3, 2, 3, 2, 4, 0, 2, 2, 3, 4, 1, 2, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2,
        2, 3, 2, 4, 2, 3, 4, 3, 2, 2, 3, 2, 3, 2, 2, 1, 3, 5, 2, 2, 2, 3, 3, 2,
        4, 2, 4, 3, 3, 2, 1, 3, 3, 3, 4, 4, 3, 3, 3, 3, 2, 3, 3, 3, 2, 4, 4, 3,
        2, 5, 4, 4, 2, 2, 3, 4, 4, 3, 3, 3, 4, 2, 4, 3, 2, 2, 4, 4, 2, 3, 3, 2,
        3, 3, 3, 4, 4, 2, 2, 3, 2, 4, 3, 2, 2, 5, 3, 3, 3, 3, 2, 2, 3, 2, 1, 2,
        3, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 4, 2, 2, 1, 3, 2, 2,
        2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2])

In [23]:
label - np.argmax(output,axis=1)

tensor([-1, -1,  1,  0,  0,  0,  1, -2,  0,  0,  0,  1, -1,  0,  1, -1,  0,  0,
         0, -1, -1,  0,  2, -1,  0,  0,  0,  1,  0,  0,  1,  0, -1,  0,  0, -1,
         1, -1,  0, -1,  0,  1,  0,  0, -1,  1,  0,  0,  1,  0,  1,  0,  0,  0,
        -1,  0,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,
         0,  2,  1,  1,  0,  0,  0,  2,  0,  0,  0,  0,  1, -2,  1, -1,  0,  0,
         1,  1, -1,  0,  1, -1,  0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0,
        -1,  2,  0,  0,  0,  1,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1, -1, -1,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0])